In [29]:
import numpy as np;
import pandas as pd;

In [30]:
## Parameters and Global variables

ROOT_DIR = "/Users/sreeramh/Documents/Kerala_Covid/SpatioTemporal/gitsrc/SpatioTemporalSEIR"

cw = 0.2

In [34]:
initDataDF = pd.read_csv(ROOT_DIR + "/data/tvm_init_data.csv")
r = len(initDataDF.index)


tData = pd.read_csv(ROOT_DIR + "/output/tmatrix_results.csv")

In [35]:
tData.head(5)
# TODO fix the unnamed colum

,Unnamed: 0,iName,jName,Tij
0,0,Amboori,Amboori,15760.800000
1,1,Amboori,Anad,0.056256
2,2,Amboori,Anchuthengu,0.004837
3,3,Amboori,Andoorkonam,0.012198
4,4,Amboori,Aruvikkara,0.127005


In [36]:
tMatrix = tData['Tij'].to_numpy().reshape(r-1,r-1)
tMatrix

array([[1.57608000e+04, 5.62561420e-02, 4.83713128e-03, ...,
        4.82911933e-03, 2.95738887e-01, 1.40273977e-01],
       [2.74115774e-02, 3.13701300e+04, 2.43349031e-02, ...,
        2.24231395e-02, 1.47105421e+00, 1.17482995e-01],
       [5.41109497e-03, 8.42760971e-02, 1.72220400e+04, ...,
        3.16959744e+01, 2.13431661e-02, 1.81703260e-02],
       ...,
       [6.25540374e-03, 9.74259400e-02, 3.04892500e+01, ...,
        1.85169600e+04, 2.46734019e-02, 2.10054944e-02],
       [5.79851016e-02, 1.67615286e-01, 2.50268834e-02, ...,
        2.60009301e-02, 3.58498800e+04, 1.69203850e+02],
       [3.77063808e-02, 9.57574317e-02, 1.95169714e-02, ...,
        2.00019435e-02, 5.05254907e+01, 3.14433900e+04]])

In [37]:
initDataDF.head(5)

,name,S,E,I,H,R
0,Amboori,15920,0,0,0,0
1,Anad,31687,0,0,0,0
2,Anchuthengu,17396,0,0,0,0
3,Andoorkonam,30781,0,0,0,0
4,Aruvikkara,33396,0,0,0,0


In [38]:
def getCij(i, j, tMatrix, initDataDF):
    Ni = initDataDF.iloc[i]['S']
    cwSum = 0
    #print("Ni = ", Ni)
    for k in range(0,r-1):
        Tik = tMatrix[i][k]
        #print("Tik = ", Tik)

        Tjk = tMatrix[j][k]
        #print("Tjk = ", Tjk)

        TlkSum = 0;
        for l in range(0, r-1):
            TlkSum = TlkSum + tMatrix[l][k]
        
        
        #print("TlkSum = ", TlkSum)
        cwSum = cwSum + (Tik/Ni)*(Tjk/TlkSum)
    return cw*cwSum

In [39]:
getCij(0,1, tMatrix, initDataDF)

1.0469799123457208e-06

In [40]:
resultsDF = pd.DataFrame(columns=['iName', 'jName', 'Cij'], );
for i in range(0,r-1):
    for j in range(0,r-1):
        Tij = getCij(i,j,tMatrix, initDataDF)
        resultsDF = resultsDF.append({'iName': initDataDF.iloc[i]['name'], 'jName': initDataDF.iloc[j]['name'], 'Cij': Tij},ignore_index=True)
 

In [41]:
resultsDF.to_csv(ROOT_DIR + "/output/cmatrix_results.csv")

In [42]:
resultsDF.head(10)

,iName,jName,Cij
0,Amboori,Amboori,1.961372e-01
1,Amboori,Anad,1.046980e-06
2,Amboori,Anchuthengu,1.287748e-07
3,Amboori,Andoorkonam,3.613491e-07
4,Amboori,Aruvikkara,2.128723e-06
5,Amboori,Aryanad,1.678061e-05
6,Amboori,Aryancodu,1.012602e-04
7,Amboori,Athiyannoor,2.018563e-06
8,Amboori,Attingal(M),4.748130e-07
9,Amboori,Azhoor,2.850022e-07
